# 1) Prodcue distance raster from point data

## 1.1 points inside 100m grid

In [1]:

## point to overlay with gridcell


import geopandas as gpd

# Load the polygon and point layers
polygons = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
points =   gpd.read_file(r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\sammelstellen\Altstoffsammelstellen_BIO_und_Mistplaetze_points_31256.shp')

output_grid_cell_with_points_inside = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp'


# Perform spatial join to find polygons covered by points
selected_polygons = gpd.sjoin(polygons, points, how="inner", op="contains")

# Display or manipulate the selected polygons as needed
print(selected_polygons.head())

# Save the selected polygons to a new shapefile if necessary
selected_polygons.to_file(output_grid_cell_with_points_inside)

print ("done")


c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_33204\2131751877.py:20: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_polygons.to_file(output_grid_cell_with_points_inside)


        EAST     NORTH                         GRID100mID  \
4465 -9300.0  340500.0  EPSG31256_GRID_100m_E-9300N340500   
5622 -8800.0  341200.0  EPSG31256_GRID_100m_E-8800N341200   
7918 -7800.0  340800.0  EPSG31256_GRID_100m_E-7800N340800   
8841 -7400.0  341100.0  EPSG31256_GRID_100m_E-7400N341100   
8993 -7300.0  333300.0  EPSG31256_GRID_100m_E-7300N333300   

                                               geometry  index_right  BEZIRK  \
4465  POLYGON ((-9300.000 340500.000, -9300.000 3406...          488    14.0   
5622  POLYGON ((-8800.000 341200.000, -8800.000 3413...          454    14.0   
7918  POLYGON ((-7800.000 340800.000, -7800.000 3409...          906    14.0   
8841  POLYGON ((-7400.000 341100.000, -7400.000 3412...          445    14.0   
8993  POLYGON ((-7300.000 333300.000, -7300.000 3334...          891    23.0   

                   STRASSE      BEZUG   ONR   BIS  ...  \
4465        Mühlbergstraße  gegenüber    44  None  ...   
5622           Hauptstraße      nebe

# 1.2 raster from the resulting grids

In [4]:
from osgeo import gdal, ogr

# Define pixel_size and NoData value of new raster
pixel_size = 100
NoData_value = -9999
# Filename of input OGR file
vector_fn  =  r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m.shp'
# Define the desired output raster shape (e.g., 1000x1000 pixels)
raster_fn =   r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Bio_sammel_mistplaetze_100m_2.tif'



# Open the data source and read in the extent
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()


#x_min, x_max, y_min, y_max = source_layer.GetExtent()
x_min=	-11200
y_min=	331000
x_max=	18800
y_max=	354000


# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[0])

print ("done")

done
